# IDS projekt

## 1. Data


In [33]:
import os
# If running on Google Colab uncomment the following lines
# and ensure you have the contents of the unzipped ids_projekt
# in your drive

#from google.colab import drive
#drive.mount('/content/drive/')


#print("moving to 'My Drive'")
#os.chdir("drive/My Drive/")

### 1.1 Exploring raw data

In [34]:
#
project_dir = "ids_projekt"
files = os.listdir(project_dir)
files.sort()


general_data_dir = files[0]
files = files[1:]


files[:5]


['tasutud_maksud_2020_i_kvartal_eng.csv',
 'tasutud_maksud_2020_ii_kvartal_eng.csv',
 'tasutud_maksud_2020_iii_kvartal_eng.csv',
 'tasutud_maksud_2020_iv_kvartal_eng.csv',
 'tasutud_maksud_2021_i_kvartal_eng.csv']

In [35]:
# Looking into I quarter 2020
i_quarter_2020 = files[0]
!pip install chardet
import chardet

# Read a small chunk of the file to analyze its encoding
for file in files:
  with open(f"{project_dir}/{file}", 'rb') as file:
    print(f"Detecting encoding for {files}")
    raw_data = file.read(10000)  # Read the first 10,000 bytes
    result = chardet.detect(raw_data)
    encoding = result['encoding']
    print("Detected encoding:", encoding)

Detecting encoding for ['tasutud_maksud_2020_i_kvartal_eng.csv', 'tasutud_maksud_2020_ii_kvartal_eng.csv', 'tasutud_maksud_2020_iii_kvartal_eng.csv', 'tasutud_maksud_2020_iv_kvartal_eng.csv', 'tasutud_maksud_2021_i_kvartal_eng.csv', 'tasutud_maksud_2021_ii_kvartal_eng.csv', 'tasutud_maksud_2021_iii_kvartal_eng.csv', 'tasutud_maksud_2021_iv_kvartal_eng.csv', 'tasutud_maksud_2022_i_kvartal_eng.csv', 'tasutud_maksud_2022_ii_kvartal_eng.csv', 'tasutud_maksud_2022_iii_kvartal_eng.csv', 'tasutud_maksud_2022_iv_kvartal_eng.csv', 'tasutud_maksud_2023_i_kvartal_eng.csv', 'tasutud_maksud_2023_ii_kvartal_eng.csv', 'tasutud_maksud_2023_iii_kvartal_eng.csv', 'tasutud_maksud_2023_iv_kvartal_eng.csv', 'tasutud_maksud_2024_i_kvartal_eng.csv', 'tasutud_maksud_2024_ii_kvartal_eng.csv', 'tasutud_maksud_2024_iii_kvartal_eng.csv']
Detected encoding: ISO-8859-1
Detecting encoding for ['tasutud_maksud_2020_i_kvartal_eng.csv', 'tasutud_maksud_2020_ii_kvartal_eng.csv', 'tasutud_maksud_2020_iii_kvartal_eng.csv'

In [36]:
# Reading data
import pandas as pd

file_params = {
    'encoding': 'ISO-8859-1',
    'sep': ';'
}

df_i_2020 = pd.read_csv(f"{project_dir}/{i_quarter_2020}", **file_params)
df_i_2020.head()

,Registry code,Name,Type,Registered in national VAT register,"Field of activity in EMTAK (Estonian Classification of Economic Activities), which is marked with an alphabetic code in the system of EMTAK",County,State taxes,Labour taxes and payments,Turnover,Number of employees
0,10000018,AMSERV AUTO AS,Company,yes,WHOLESALE AND RETAIL TRADE; REPAIR OF MOTOR VE...,Harju ( Tallinn ),"635 586,56","669 944,03",NaN,208.0
1,10000024,"EESTI RAAMAT, OÜ",Company,yes,INFORMATION AND COMMUNICATION,Harju ( Tallinn ),"22 847,67","14 375,33","129 993,24",10.0
2,10000062,ALDO KOPPEL,Self-employed person,yes,"AGRICULTURE, FORESTRY AND FISHING",Ida-Viru ( Lüganuse vald ),NaN,NaN,"185,00",NaN
3,10000127,"ARAVETE APTEEK, TÜ",Company,yes,WHOLESALE AND RETAIL TRADE; REPAIR OF MOTOR VE...,Järva ( Järva vald ),"8 717,01","4 503,35","116 415,20",2.0
4,10000165,"KIVIÕLI KAUBAHOOV, AS",Company,yes,WHOLESALE AND RETAIL TRADE; REPAIR OF MOTOR VE...,Ida-Viru ( Lüganuse vald ),"90 795,72","57 953,73","842 647,07",45.0


In [37]:
df_i_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142890 entries, 0 to 142889
Data columns (total 10 columns):
 #   Column                                                                                                                                       Non-Null Count   Dtype  
---  ------                                                                                                                                       --------------   -----  
 0   Registry code                                                                                                                                142890 non-null  object 
 1   Name                                                                                                                                         142890 non-null  object 
 2   Type                                                                                                                                         142890 non-null  object 
 3   Registered in national VAT re

In [38]:
# Different entity types
df_i_2020['Type'].value_counts()

Type
Company                                          119920
Non-profit association                            12375
Self-employed person                               4148
Non-resident                                       4047
Government or state authority                      1925
Foundation                                          473
International organisation located in Estonia         2
Name: count, dtype: int64

In [39]:
# Example of different types of legal entities in Estonia
df_i_2020.groupby('Type').first()

,Registry code,Name,Registered in national VAT register,"Field of activity in EMTAK (Estonian Classification of Economic Activities), which is marked with an alphabetic code in the system of EMTAK",County,State taxes,Labour taxes and payments,Turnover,Number of employees
Type,,,,,,,,,
Company,10000018,AMSERV AUTO AS,yes,WHOLESALE AND RETAIL TRADE; REPAIR OF MOTOR VE...,Harju ( Tallinn ),"635 586,56","669 944,03","129 993,24",208.0
Foundation,90000050,ESTONIAN-REVELIA ACADEMIC FUND'I SIHTASUTUS EE...,no,OTHER SERVICE ACTIVITIES,Harju ( Tallinn ),"198,75","198,75","23 548,70",1.0
Government or state authority,70000042,PÕLLUMAJANDUSUURINGUTE KESKUS,yes,"PROFESSIONAL, SCIENTIFIC AND TECHNICAL ACTIVITIES",Harju ( Saku vald ),"1 073,40","0,00","843,12",167.0
International organisation located in Estonia,98000012,BALTI KAITSEKOLLEDÞ,no,EDUCATION,Tartu ( Tartu linn ),"187 846,92","197 796,97",None,38.0
Non-profit association,80000037,EESTI SUURPEREDE ABISTAMISE SELTS (LIKVIDEERIM...,no,OTHER SERVICE ACTIVITIES,Harju ( Tallinn ),"307,41","321,42","42 395,00",1.0
Non-resident,60000066,LEDVANCE OY,yes,WHOLESALE AND RETAIL TRADE; REPAIR OF MOTOR VE...,Harju ( Tallinn ),"186 973,21","13 640,58","1 588 190,12",2.0
Self-employed person,10000062,ALDO KOPPEL,yes,"AGRICULTURE, FORESTRY AND FISHING",Ida-Viru ( Lüganuse vald ),"1 308,94","691,44","185,00",1.0


In [40]:
# Some outliers that don't really fit our purposes
df_i_2020[df_i_2020['Type'] == 'International organisation located in Estonia']

,Registry code,Name,Type,Registered in national VAT register,"Field of activity in EMTAK (Estonian Classification of Economic Activities), which is marked with an alphabetic code in the system of EMTAK",County,State taxes,Labour taxes and payments,Turnover,Number of employees
142766,98000012,BALTI KAITSEKOLLEDÞ,International organisation located in Estonia,no,EDUCATION,Tartu ( Tartu linn ),"187 846,92","197 796,97",NaN,38.0
142767,98000026,KOOPERATIIVNE KÜBERKAITSE KOMPETENTSIKESKUS,International organisation located in Estonia,no,NaN,Harju ( Tallinn ),"2 012,78","2 182,18",NaN,NaN


In [41]:
# Employee count data overview for each type of legal entity
df_i_2020.groupby('Type').agg(
    min_employees=('Number of employees', 'min'),
    max_employees=('Number of employees', 'max'),
    #range_employees=('Number of employees', lambda x: x.max() - x.min()),
    null_count=('Number of employees', lambda x: x.isnull().sum())
)

,min_employees,max_employees,null_count
Type,,,
Company,1.0,3738.0,40042
Foundation,1.0,4537.0,53
Government or state authority,1.0,4527.0,12
International organisation located in Estonia,38.0,38.0,1
Non-profit association,1.0,291.0,2680
Non-resident,1.0,331.0,3486
Self-employed person,1.0,18.0,2745


In [42]:
# Self-employed person legal entity type employee counts
df_i_2020[df_i_2020['Type'] == 'Self-employed person'].value_counts('Number of employees')

Number of employees
1.0     895
2.0     280
3.0     104
4.0      53
5.0      20
6.0      18
8.0       8
7.0       6
11.0      5
9.0       4
13.0      3
15.0      2
10.0      1
12.0      1
16.0      1
17.0      1
18.0      1
Name: count, dtype: int64

### 1.2 Initial thoughts

1. Moving forward should bring in the [open company data](https://avaandmed.ariregister.rik.ee/en/downloading-open-data)
2. Registry code can be used to identify companies even if they have changed name, maybe namechange is an indicator if company is gonna do well or not :D ?
3. Should we exclude estonian `FIE` (Self-Employed Person) since that's not what we are looking for?
4. Should have functions that give us parsed data, eg. We want to test our predictions on 3rd quarter 2024 so we read in data till the prev quarter?
5. Should we normalize and adjust things for inflation?
6. What more data can we include, eg. last company name change, median number of employees or sliding window employee change?
7. What else intresting can we find, maybe court cases, maybe business licences eg. alcohol sales?


### 1.3 Making our dataset


In [52]:
import pandas as pd

import chardet

def get_file_encoding(file_name):
  with open(f"{project_dir}/{file_name}", 'rb') as file:
      raw_data = file.read(10000)  # Read the first 10,000 bytes
      result = chardet.detect(raw_data)
      encoding = result['encoding']
  return encoding

def file_name_for_quarter_year(quarter, year):
  def quarter_to_string(quarter):
    if quarter == 1:
      return "i"
    if quarter == 2:
      return "ii"
    if quarter == 3:
      return "iii"
    if quarter == 4:
      return "iv"
    raise ValueError(f"Invalid quarter value: {quarter}")

  assert quarter in (1, 2, 3, 4)
  assert year in range(2020, 2024 + 1), f"Year must be between 2020 and 2024"

  return f"tasutud_maksud_{year}_{quarter_to_string(quarter)}_kvartal_eng.csv"

def get_data_for_quarter_year(quarter, year):
  file_name = file_name_for_quarter_year(quarter, year)

  file_params = {
    'encoding': get_file_encoding(file_name),
    'sep': ';'
  }
  assert quarter in (1, 2, 3, 4)
  assert year in range(2020, 2024 + 1), f"Year must be between 2020 and 2024"
  if year == 2024:
    assert quarter in (1, 2), f"Quarter must be 1 or 2 for year 2024, since we dont have \
    next quarter labour taxes and payments for q3"

  df = pd.read_csv(f"{project_dir}/{file_name}", **file_params)
  # We filter only the types of entities that have to provide needed info quarterly
  types_to_keep = ['Company', 'Non-profit association', 'Government or state authority']
  df = df[df['Type'].isin(types_to_keep)]

  # Renaming for simplicity

  df.rename(columns={
      "Number of employees": "Employees"
  }, inplace=True)

  # We only want the following columns
  df = df[["Registry code", "Name", "Type", "Turnover", "Employees", "Labour taxes and payments"]]

  # Add info field about data
  df['Info'] = f"Data for Q{quarter} {year}"

  # Adding next quarter labour taxes and payments
  next_quarter, next_quarter_year = ((quarter + 1, year), (1, year + 1))[quarter == 4]
  next_quarter_file_name = file_name_for_quarter_year(next_quarter, next_quarter_year)
  next_quarter_file_params = {
    'encoding': get_file_encoding(next_quarter_file_name),
    'sep': ';'
  }
  next_quarter_df = pd.read_csv(f"{project_dir}/{next_quarter_file_name}", **next_quarter_file_params)
  next_quarter_df.rename(columns={
      "Number of employees": "Employees"
  }, inplace=True)

  df = pd.merge(df,
                next_quarter_df[['Registry code', 'Employees', 'Labour taxes and payments']],
                on='Registry code',
                how='left',
                suffixes=('', '_next_quarter')
                )
  return df

general_data = None

def include_general_data(quarterly_df):
  global general_data
  if general_data is None:
    general_data_full_dir = f"{project_dir}/{general_data_dir}"
    print(general_data_full_dir) # ids_projekt/general_data
    
    files = os.listdir(general_data_full_dir)
    print("Files found:", files)
    [general_data_file] = files


    general_data = pd.read_parquet(f"{general_data_full_dir}/{general_data_file}")
    # registry_codes = list(quarterly_df['Registry code'].unique())
    # return df_general_data[df_general_data['ariregistri_kood'].isin(registry_codes)]
    general_data['Registry code'] = general_data['ariregistri_kood'].astype(str)

  quarterly_df['Registry code'] = quarterly_df['Registry code'].astype(str)
  df_all = pd.merge(quarterly_df, general_data, on='Registry code', how='left',
                  suffixes=('', ' from genral data'))

  df_all = df_all[df_all['yldandmed'].notna()]
  return df_all


#df = get_data_for_quarter_year(2, 2024)
#df = include_general_data(df)
df = None


In [47]:
import numpy as np

def get_esmaregistreerimise_kpv(yldandmed):
  return yldandmed['esmaregistreerimise_kpv']
get_ettevõtte_staatus = lambda x: x['staatus_tekstina']

def get_ettevõtte_põhitegevusala(yldandmed):
  try:
    [pohitegevusala] = [item for item in yldandmed["teatatud_tegevusalad"] if item['on_pohitegevusala']]
    return pohitegevusala['emtak_kood']
  except:
    return None
  return None


def yldandmed_serializer(obj):
    if isinstance(obj, np.ndarray):  # Convert ndarray to list
        return obj.tolist()
    raise TypeError(f"Object of type {type(obj).__name__} is not JSON serializable")

def get_ettevõtte_põhikapital(yldandmed):
  kapitalid = yldandmed['kapitalid']
  if kapitalid == None:
    return kapitalid
  if len(kapitalid) == 1:
    return kapitalid[0]['kapitali_suurus']
  return None

def finalize_data(data):
  data_copy = data.copy()
  data_copy["Asutamise kuupäev"] = data["yldandmed"].apply(get_esmaregistreerimise_kpv)
  data_copy["Staatus"] = data['yldandmed'].apply(get_ettevõtte_staatus)
  data_copy["Põhitegevusala"] = data['yldandmed'].apply(get_ettevõtte_põhitegevusala)
  data_copy["Põhikapital"] = data['yldandmed'].apply(get_ettevõtte_põhikapital)
  return data_copy

def reorder_columns(df):
  ordered_columns = [
      'Name',
      'Registry code',
      'Info',
      'Type',
      'Turnover',
      'Employees',
      'Asutamise kuupäev',
      'Staatus',
      'Põhitegevusala',
      'Põhikapital',
      'Labour taxes and payments',
      'Employees_next_quarter',
      'Labour taxes and payments_next_quarter']
  return df[ordered_columns]
df = None


     ---------------------------------------- 0.0/467.2 kB ? eta -:--:--
     - ----------------------------------- 20.5/467.2 kB 330.3 kB/s eta 0:00:02
     ---- -------------------------------- 61.4/467.2 kB 825.8 kB/s eta 0:00:01
     ----------------------------- -------- 368.6/467.2 kB 3.3 MB/s eta 0:00:01
     -------------------------------------- 467.2/467.2 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ------- -------------------------------- 0.4/2.2 MB 12.6 MB/s eta 0:00:01
   ---------------- ----------------------- 0.9/2.2 MB 11.0 MB/s eta 0:00:01
   -------------------------- ------------- 1.4/2.2 MB

In [48]:
def get_compiled_data(quarter, year):
  df = get_data_for_quarter_year(quarter, year)
  df = include_general_data(df)
  df = finalize_data(df)
  df = reorder_columns(df)
  return df

## Preprocessing


In [53]:
df_train = get_compiled_data(2, 2024)
df_train.head()

ids_projekt/general_data
Files found: ['13_11_yldandmed.parquet']


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NAType'

In [ ]:
df_train.info()

In [ ]:
from datetime import datetime

def preprocess(data, debug=0):
  def drop_missing_rows(d):
    return d.dropna(subset=['Employees', 'Labour taxes and payments', 'Employees_next_quarter', 'Labour taxes and payments_next_quarter'])
  def numeric_string_to_float(value):
    if type(value) == str:
      value = value.replace(',', '.')
      value = value.replace(" ", "")
      assert " " not in value, f"Found whitespace in value {value}"
    try:
      return float(value)
    except:
      return float(0)
  def none_to_zero(value):
    if value == None:
      return 0
    return value
  today = datetime.today()
  def labour_taxes_and_payments_to_avg_gross_salary(labour_taxes_and_payments, employees):
    coef = 0.5668
    # We consider that 25% of estonians don't gather funded pension
    coef_without_funded_pension = coef - 0.02 + 0.015
    coef = coef_without_funded_pension

    assert type(labour_taxes_and_payments) == float, f"labour_taxes_and_payments is not float, but {type(labour_taxes_and_payments)}"
    employees = int(numeric_string_to_float(employees))
    assert type(employees) == int, f"employees is not int, but {type(employees)}"

    gross_per_employee_per_quarter = labour_taxes_and_payments / coef / employees
    gross_per_month = gross_per_employee_per_quarter / 3
    return round(gross_per_month, 2)
  def taxes_per_month_per_emp(taxes, employees):
    return round(taxes / employees / 3, 2)
  def founding_date_to_categoric(date_str):
      date_format = "%d.%m.%Y"
      given_date = datetime.strptime(date_str, date_format)

      age_years = (today - given_date).days / 365.25

      if age_years < 3:
          return "less than 3 years"
      if 3 <= age_years < 5:
          return "3-5 years"
      if 5 <= age_years < 10:
          return "5-10 years"
      if 10 <= age_years < 20:
          return "10-20 years"
      return "20+ years"
  def p6hikapital_to_categoric(value):
    value = none_to_zero(value)
    value = numeric_string_to_float(value)
    if value < 0.005:
      return "0 €"
    if value < 10000:
     return "< 10k €"
    if value < 100000:
      return "< 100k €"
    return "100k € +"
  data_copy = data.copy()
  if debug:
    print("Length before:", len(data_copy))
  data_copy = drop_missing_rows(data_copy)
  if debug:
    print("Length after dropping missing rows:", len(data_copy))


  data_copy['Capital'] = data_copy['Põhikapital'].apply(p6hikapital_to_categoric)
  data_copy.drop(columns=['Põhikapital'], inplace=True)

  data_copy['Employees'] = data_copy['Employees'].fillna(0)
  data_copy['Employees'] = data_copy['Employees'].apply(numeric_string_to_float)

  data_copy['Turnover'] = data_copy['Turnover'].fillna(0)
  data_copy['Turnover'] = data_copy['Turnover'].apply(numeric_string_to_float)

  # We apply none to zero, since nan-s were explicit in dataset, we believe
  # 0 values were written as empty strings and got written as None
  data_copy['Labour taxes and payments'] = data_copy['Labour taxes and payments'].apply(none_to_zero)
  data_copy['Labour taxes and payments'] = data_copy['Labour taxes and payments'].apply(numeric_string_to_float)

  data_copy['Labour taxes and payments_next_quarter'] = data_copy['Labour taxes and payments_next_quarter'].apply(none_to_zero)
  data_copy['Labour taxes and payments_next_quarter'] = data_copy['Labour taxes and payments_next_quarter'].apply(numeric_string_to_float)
  # Company capital, really only applicaple for `Type = Company`
  # Make the age of the company a gategorical value
  data_copy['Company age'] = data_copy['Asutamise kuupäev'].apply(founding_date_to_categoric)
  data_copy.drop(columns=['Asutamise kuupäev'], inplace=True)

  # Avg employee salary is more meaningful, let's calculate it from the taxes
  data_copy['Avg monthly salary'] = data_copy.apply(lambda row: labour_taxes_and_payments_to_avg_gross_salary(row['Labour taxes and payments'], row['Employees']), axis=1)
  data_copy.drop(columns=['Labour taxes and payments', 'Employees'], inplace=True)

  data_copy['Avg next quarter month salary'] = data_copy.apply(lambda row: labour_taxes_and_payments_to_avg_gross_salary(row['Labour taxes and payments_next_quarter'], row['Employees_next_quarter']), axis=1)
  data_copy.drop(columns=['Labour taxes and payments_next_quarter', 'Employees_next_quarter'], inplace=True)



  # The problem we realized too late was that in case of every 4th month, the previous months employee count is
  # reported in previous quarter, but the taxes become payable for the next quarter.
  # We don't have access to this monthly info to make corrections...
  # In case a project that means many employees ends on last month of the quarter, the taxes still have to be paid
  # next quarter but the employee count can drop significantly.
  # This is also the same in case of major layoffs.

  # For this problem, we decided to drop all values where the monthly avg gross salary for employee changes by more than 5k
  data_copy = data_copy[(data_copy['Avg next quarter month salary'] - data_copy['Avg monthly salary']) <= 5000]
  if debug:
    print("Length after dropping avg salary changes:", len(data_copy))
  # Also we are dropping all values where we get a calculated salary over 20k
  data_copy = data_copy[data_copy['Avg monthly salary'] <= 20000]
  if debug:
    print("Length after dropping avg salary over 20k:", len(data_copy))
  # and where predicted salary is over 20k
  data_copy = data_copy[data_copy['Avg next quarter month salary'] <= 20000]
  if debug:
    print("Length after dropping predicted salary over 20k:", len(data_copy))


  return data_copy




#df_train_preprocessed = preprocess(df_train)
#print("Length:", len(df_train_preprocessed))
#df_train_preprocessed
#df_train_preprocessed[df_train_preprocessed['Avg monthly salary'] < 1000]

In [ ]:
from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
# Apply preprocessing

preprocess_params = dict(
    debug=True # default False
)

df_train_preprocessed = preprocess(df_train, **preprocess_params)

df_train_preprocessed.drop(columns=['Name', 'Registry code', 'Info'], inplace=True)


X_data = df_train_preprocessed.drop(columns=['Avg next quarter month salary'])
y_data = df_train_preprocessed['Avg next quarter month salary']

categorical_cols = X_data.select_dtypes(include=['object']).columns
numerical_cols = X_data.select_dtypes(include=['int64', 'float64']).columns

# One-hot encode categorical variables and scale numerical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ]
)
print(df_train_preprocessed.columns)

#df_train_preprocessed['salary_change'] = df_train_preprocessed['Avg next quarter month salary'] - df_train_preprocessed['Avg monthly salary']

# Filter rows where the change is greater than n
#df_salary_change_more_than = df_train_preprocessed[(df_train_preprocessed['Avg next quarter month salary'] - df_train_preprocessed['Avg monthly salary']) > 5000]

#df_salary_change_more_than['Type'].value_counts()

In [ ]:
### Making dataframe for all historical data
preprocess_params = dict(
    debug=True # default False
)
def preprocess_all(quarter = 1, year = 2020, **kwargs):
  year = 2020
  quarter = 1
  df = pd.DataFrame()
  while year < 2024 or quarter < 3:
    if preprocess_params['debug']:
      print(f"Processing {quarter} {year}")
    df_for_quarter = get_compiled_data(quarter, year)
    df_preproccessed_1 = preprocess(df_for_quarter, **preprocess_params)
    df_preproccessed_1['Year'] = str(year)
    df_preproccessed_1['Quarter'] = str(quarter)
    df = pd.concat([df, df_preproccessed_1])
    if quarter == 4:
      quarter = 1
      year += 1
    else:
      quarter += 1
  return df
df_historical = preprocess_all(1, 2020, **preprocess_params)

all_data = df_historical.copy()
df_historical.drop(columns=['Name', 'Registry code', 'Info'], inplace=True)
df_historical.info()


In [ ]:

y_data

## Testing models with previous quarter data

In [ ]:
# Predicting from previous quarter
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor

from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, accuracy_score, mean_squared_error




X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)


random_forest_params = {
    'n_estimators': 100, # default 100
    'max_depth': 30, # default None
    'min_samples_split': 8, # default 2
    'min_samples_leaf': 1, # default 1
    'max_features': 'sqrt', # defaul 1.0
    'n_jobs': -1, # use all cores
    'random_state': None # default None
}

gradient_boosting_params = dict(
    loss='squared_error',
    learning_rate=0.1,
    n_estimators=100, #default 100
    subsample=1.0,
    criterion='friedman_mse',
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_depth=3,
    min_impurity_decrease=0.0,
    init=None,
    random_state=None,
    max_features=None,
    alpha=0.9,
    verbose=0,
    max_leaf_nodes=None,
    warm_start=False,
    validation_fraction=0.1,
    n_iter_no_change=None,
    tol=0.0001,
    ccp_alpha=0.0
)

xgboost_params = dict(
    objective='reg:squarederror',
    learning_rate=0.05, # default 0.1
    n_estimators=1000, #default 100
    #subsample=1.0,
    #max_depth=3,
    #init=None,
    #random_state=None,
    #max_features=None,
    #alpha=0.9,
    #max_leaf_nodes=None,
    #n_iter_no_change=None
    n_jobs = -1
)

models = {
    'Simple linear': Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', LinearRegression())
    ]),
    'Ridge Regression': Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', Ridge(alpha=1.0))
    ]),
    'Random Forest': Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', RandomForestRegressor(**random_forest_params))
    ]),
    'Gradient Boosting': Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', GradientBoostingRegressor(**gradient_boosting_params))
    ]),
    'XGBoost': Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', XGBRegressor(**xgboost_params))
    ])
}



model_results = dict()
model_preds = dict()

for name, model in models.items():
    print(f"Training {name}")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    rmse = np.sqrt(mse)
    print(f"Root Mean Squared Error: {rmse}")


    model_results[name] = rmse
    model_preds[name] = y_pred

    #for test, pred in zip(y_test, y_pred):
    #  print(f"Test: {test}, Predicted: {pred}")

for name, y_pred in model_preds.items():
    plt.scatter(y_test, y_pred)
    plt.title(f'Predictions for {name}')
    plt.xlabel('Actual Labor Taxes')
    plt.ylabel('Predicted Labor Taxes')
    plt.show()
    print()



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Convert the dictionary to a DataFrame for easier plotting with seaborn
df = pd.DataFrame(list(model_results.items()), columns=['Model', 'RMSE'])

# Create the barplot
plt.figure(figsize=(10, 6))
sns.barplot(y='RMSE', x='Model', data=df, palette='viridis')

# Set the title and labels
plt.title('Model RMSE Comparison when predicting from previous quarter', fontsize=16)
plt.xlabel('Root Mean Squared Error (RMSE)', fontsize=12)
plt.ylabel('Model name', fontsize=12)

# Display the plot
plt.show()

## Testing models with historical data

In [ ]:
# Using historical data


X_data = df_historical.drop(columns=['Avg next quarter month salary'])
y_data = df_historical['Avg next quarter month salary']

categorical_cols = X_data.select_dtypes(include=['object']).columns
numerical_cols = X_data.select_dtypes(include=['int64', 'float64']).columns

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)


random_forest_params = {
    'n_estimators': 100, # default 100
    'max_depth': 30, # default None
    'min_samples_split': 8, # default 2
    'min_samples_leaf': 1, # default 1
    'max_features': 'sqrt', # defaul 1.0
    'n_jobs': -1, # use all cores
    'random_state': None # default None
}

gradient_boosting_params = dict(
    loss='squared_error',
    learning_rate=0.1,
    n_estimators=100, #default 100
    subsample=1.0,
    criterion='friedman_mse',
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_depth=3,
    min_impurity_decrease=0.0,
    init=None,
    random_state=None,
    max_features=None,
    alpha=0.9,
    verbose=0,
    max_leaf_nodes=None,
    warm_start=False,
    validation_fraction=0.1,
    n_iter_no_change=None,
    tol=0.0001,
    ccp_alpha=0.0
)

xgboost_params = dict(
    objective='reg:squarederror',
    learning_rate=0.05, # default 0.1
    n_estimators=1000, #default 100
    #subsample=1.0,
    #max_depth=3,
    #init=None,
    #random_state=None,
    #max_features=None,
    #alpha=0.9,
    #max_leaf_nodes=None,
    #n_iter_no_change=None
    n_jobs = -1
)

models = {
    'Simple linear': Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', LinearRegression())
    ]),
    'Ridge Regression': Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', Ridge(alpha=1.0))
    ]),
    'Random Forest': Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', RandomForestRegressor(**random_forest_params))
    ]),
    'Gradient Boosting': Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', GradientBoostingRegressor(**gradient_boosting_params))
    ]),
    'XGBoost': Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', XGBRegressor(**xgboost_params))
    ])
}


model_results = dict()
model_preds = dict()

for name, model in models.items():
    print(f"Training {name}")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    rmse = np.sqrt(mse)
    print(f"Root Mean Squared Error: {rmse}")


    model_results[name] = rmse
    model_preds[name] = y_pred

    #for test, pred in zip(y_test, y_pred):
    #  print(f"Test: {test}, Predicted: {pred}")

for name, y_pred in model_preds.items():
    plt.scatter(y_test, y_pred)
    plt.title(f'Predictions for {name}')
    plt.xlabel('Actual Labor Taxes')
    plt.ylabel('Predicted Labor Taxes')
    plt.show()
    print()



In [ ]:

# Convert the dictionary to a DataFrame for easier plotting with seaborn
df = pd.DataFrame(list(model_results.items()), columns=['Model', 'RMSE'])

# Create the barplot
plt.figure(figsize=(10, 6))
sns.barplot(y='RMSE', x='Model', data=df, palette='viridis')

# Set the title and labels
plt.title('Model RMSE Comparison when predicting from historic data', fontsize=16)
plt.ylabel('Root Mean Squared Error (RMSE)', fontsize=12)
plt.xlabel('Model name', fontsize=12)

# Display the plot
plt.show()

## Simple tool for users

### Finding the best model for cli tool


In [ ]:
best_model_name = None
best_res = 99999999
for name, rmse in model_results.items():
  if rmse < best_res:
    best_res = rmse
    best_model_name = name
print(f"Best model is {best_model_name} with RMSE {best_res}")

In [ ]:
# Training our best model on the whole dataset
best_model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', XGBRegressor(**xgboost_params))
    ])
X_all = df_historical.drop(columns=['Avg next quarter month salary'])
y_all = df_historical['Avg next quarter month salary']


best_model.fit(X_all, y_all)


### Playground

In [ ]:
def find_company_by_name():
  companies = set(zip(all_data['Registry code'],all_data['Name']))
  unique_companies = {reg_code: name for reg_code, name in companies}

  # Convert back to a set of tuples if needed
  companies = set(unique_companies.items())
  while True:
    user_input = input("Enter the company name or a partial name to search: ").strip()
    matches = [company for company in companies if user_input.lower() in company[1].lower()]

    if len(matches) == 1:
      print(f"Found exact match: {matches[0][1]}")
      return matches[0]
    if len(matches) == 0:
      print(f"No company with enough public info for {user_input}")
      print(f"If you'd like to enter data manually enter 1, or 0 to refine search: ")
      try:
        selection = int(input("If you'd like to enter data manually enter 1, or 0 to refine search: "))
        if selection == 0:
          continue
        if selection == 1:
          return None, user_input
      except ValueError:
        continue
    elif len(matches) > 5:
      print("Too many matches found. Please refine your search.")
    else:
      print("Here are the matching options:")
      for id, m in enumerate(matches):
        print(f"{id + 1}. {m}")
      try:
        selection = int(input("Enter the number of your choice, or 0 to refine search: "))
        if selection == 0:
          continue
        if 1 <= selection <= len(matches):
          print(f"You selected: {matches[selection - 1]}")
          return matches[selection - 1]
        print("Invalid selection. Please try again.")
      except ValueError:
        print("Invalid input. Please enter a number.")
def get_next_quarter():
  return 4, 2024

def manual_input(company_name):
  entity_type = int(input("Enter the entity type 1 - 'Company',  2- 'Government or state authority', 3 - 'Non-profit association': "))
  if entity_type == 1:
    entity_type = 'Company'
  elif entity_type == 2:
    entity_type = 'Government or state authority'
  else:
    entity_type = 'Non-profit association'

  turnover = float(input("Enter the company_turnover: "))

  capital = int(input("Enter company capital 1 - '0 €', 2 - '< 10k €', 3 - '< 100k €', 4 - '100k € +'"))
  if capital == 1:
    capital = '0 €'
  elif capital == 2:
    capital = '< 10k €'
  elif capital == 3:
    capital = '< 100k €'
  else:
    capital = '100k € +'

  company_age = int(input("Enter company age 1 - '20+ years', 2 - '10-20 years', 3 - '5-10 years', 4 - '3-5 years',\
       5 - 'less than 3 years'"))
  if company_age == 1:
    company_age = '20+ years'
  elif company_age == 2:
    company_age = '10-20 years'
  elif company_age == 3:
    company_age = '5-10 years'
  elif company_age == 4:
    company_age = '3-5 years'
  else:
    company_age = 'less than 3 years'
  last_average_salary = float(input("Enter the last average salary: "))
  emtak_code = int(input("Enter the emtak code: (for example 45111)"))
  next_quarter, year = get_next_quarter()
  manual = {
    "Type": [entity_type],
    "Turnover": [turnover],
    "Staatus": ["Registrisse kantud"],
    "Põhitegevusala": [emtak_code],
    "Capital": [capital],
    "Company age": [company_age],
    "Avg monthly salary": [last_average_salary],
    "Year": [year],
    "Quarter": [next_quarter]
  }
  return pd.DataFrame(manual).iloc[0]


reg_code, company_name = find_company_by_name()

if reg_code:
  next_quarter, year = get_next_quarter()
  all_data_sorted = all_data.sort_values(by=['Year', 'Quarter'], ascending=[False, False])
  location = all_data_sorted[all_data_sorted['Registry code'] == reg_code]
  example = location.iloc[0]
  example = example.copy()
  example['Year'] = year
  example['Quarter'] = next_quarter
else:
  example = manual_input(company_name)

prediction = best_model.predict(pd.DataFrame([example]))
print(f"Predicted avg salary for Q{example['Quarter']} {example['Year']}: {prediction[0]}")